In [1]:
import requests
import csv
from datetime import datetime
import os
import pandas as pd
import warnings
from tqdm import tqdm 
warnings.simplefilter(action='ignore', category=FutureWarning)

# ddi - Mini-Challenge zu LE3, NoSQL
Roman Studer, Simon Luder

## Use Case
<p align="center">
  <img src="./data/images/Konzept.png" alt="drawing" width="900"/>
</p>

Wir unterbreiten den Vorschlag eine NoSQL Datenbank (MongoDB) als Data Lake für die Speicherung von Time Series Daten zu verwenden und die Analyse besagter Daten auf einer Time-Series Datenbank durchzuführen. Die Analyse dieser Daten kann mittels einer auf Time Series optimierte Datenbank auf Abruf geschehen. Als Beispiel verwenden wir als Datenquelle openSenseMap, welche Messwerte und Sensormetainformationen über eine API zur Verfügung stellt. Ein aktiver Sensor sendet periodisch (je nach Sensor alle paar Sekunden oder Minuten) einen Messwert. Bei einem Intervall von 10 Sekunden sendet ein Sensor pro Jahr 3'153'600 Datenpunkte. 

OpenSenseMap erlaubt es ein ganzes Gebiet (Mittels Angabe von Breiten- und Längengrad) zu überwachen. Die Anzahl Sensoren, sowie deren Attribute kann sich über die Zeit ändern. Wenn zum Beispiel ein neuer Sensor im gleichen Gebiet in Betrieb genommen wird. Daher ist die Datenspeicherung in einer NoSQL, schemenlosen Datenbank geeignet. MongoDB ist dabei aufgrund des flexigblem Schema und einfacher horizontaler Skalierung gut geeignet. Dadurch sind wir auf Änderungen in den durch die API erhaltenen Attributen, sowie auf grosse Änderungen in der Datenmenge gewappnet. MongoDB ist allerdins nicht für die Analyse von Time Series geeignet. InfluxDB, eine Time Series Datenbank, ist für Time Series Daten optimiert und kann schnell Aggregationen über eine grosse Anzahl von Datenpunkten (über Timestamp Indexiert) durchführen. Ein Beispiel wäre ein Moving Average mit kleinem Fenster über mehrere Millionen Datenpunkte.

Weiter existiert ein MongoDB-Plugin auf Telegraf welche die Performance der MongoDB überwachen kann. Somit kann das Monitoring über Influx betrieben werden. 

Im Anschluss setzen wir sowohl eine MongoDB als auch eine InfluxDB auf. Über ein Script laden wir alle Sensordaten auf dem Gelände der ETH-Zürich, welche OpenSenseMap zur Verfügung stellt herunter und speichern diese in der MongoDB. Im Anschluss messen wir die Zeit für Aggregationen bei steigenden Datenpunkten einzeln für beide Datenbanken, sowie für den Fall wenn MongoDB als Datalake verwendet wird. Dadurch können wir einen Punkt identifizieren ab dem es nicht mehr sinnvoll ist nur mit einer MongoDB zu arbeiten, sondern die InfluxDB zur Analyse hinzuzuziehen.

### Vergleich MongoDB vs. InfluxDB

## Datenmodell

## Load Data

In [2]:
#get list of ifu boxes at eth
url = 'https://api.opensensemap.org/boxes?'
bbox = '8.50269672304309, 47.40598032642525,  8.512126181507432, 47.4113301084323 ' # boundary box around eth zurich
boxes = requests.get(url, params={'bbox':bbox, 'full':'false'}).json()

In [3]:
from_date = '2021-03-01T10:05:49.581Z'
to_date = '2021-03-03T10:05:49.581Z'
data_format = 'csv'

for box in tqdm(boxes):
    box_id = box['_id']
    box_name = box['name']
    location = box['currentLocation']['coordinates']
    lat, lon = location[0], location[1]
    for sensor in box['sensors']:
        try:
            sensor_id = sensor['_id']
            sensor_name = sensor['title']
            sensor_name.replace('/', '')
            sensor_unit = sensor['unit']
        except:
            pass
        
        #url = f'https://api.opensensemap.org/boxes/{box_id}/data/{sensor_id}?format={data_format}&download=true'
        url = f'https://api.opensensemap.org/boxes/{box_id}/data/{sensor_id}?from-date={from_date}&to-date={to_date}&download=true&format={data_format}'
        r = requests.get(url, stream=True)
        if (len(r.text) > 16): # check if sensor returns values (header has length 16)
            with open(f'./data/{box_name}_{sensor_name}.csv', 'wb') as f:
                for _, line in enumerate(r.iter_lines()):
                    if _ == 0: # define header
                        line = 'box_name,sensor_name,box_id,sensor_id,lat,lon,unit,current_time,value\n'
                    else:
                        time, value = (line.decode("utf-8").split(','))
                        time = time.replace('T', ' ').replace('Z', '')
                        time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S.%f').strftime('%Y-%m-%d %H:%M:%S')
                        line= f'{box_name},{sensor_name},{box_id},{sensor_id},{lat},{lon},{sensor_unit},{time},{value}\n'
                    f.write(line.encode())

 33%|███████████████████████████                                                       | 27/82 [00:39<01:20,  1.47s/it]


KeyboardInterrupt: 

## Setup MongoDB

In [ ]:
#!pip install pymongo
from pymongo import MongoClient
import pymongo

### Prepare data

In [ ]:
def csv_to_dict(path, file):
    '''converts a csv file to a dictionary'''
    data = pd.read_csv(path + file)
    pd.to_datetime(data.current_time, format="%Y-%m-%d %H:%M:%S")
    dictionary = dict()
    dictionary["_id"] = file.replace('.csv', '')
    dictionary["box_name"] = data["box_name"][0]
    dictionary["sensor_name"] = data["sensor_name"][0]
    dictionary["box_id"] = data["box_id"][0]
    dictionary["sensor_id"] = data["sensor_id"][0]
    dictionary["lat"] = data["lat"][0]
    dictionary["lon"] = data["lon"][0]
    dictionary["unit"] = data["unit"][0]
    dictionary["measurments"] = dict(zip(data["current_time"], data["value"]))
    return dictionary

### Create db

In [ ]:
mongo_client = pymongo.MongoClient('localhost', 27017)
mongo_db = mongo_client["ddi_mc2"]

### Populate db

In [ ]:
VERBOSE = False
path = './data/'
for file in tqdm(os.listdir("./data")):
#     print(file.split("_")[0])
    if file.endswith('.csv'): # check for filetype
        if not mongo_db[file.split("_")[0]].count_documents({"_id":file.replace('.csv', '')}) > 0:
            dictionary = csv_to_dict(path, file)
            mongo_db[file.split("_")[0]].insert_one(dictionary)
            if VERBOSE:
                print("populate:", file)

In [ ]:
mongo_db.list_collection_names()

In [ ]:
mongo_db["IfU SenseBox2021 11A"].find().distinct('_id')

## Setup InfluxDB 

In [ ]:
# !pip install influxdb-client
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
token = "AW-zLqzOTpQW4sRYaKbdXpSxBLkxT8rT-RZA-IS5MYo41RZ40YoOCoNYTyu9S2La5W4KpcDzDgCfj53fk6aZuw=="
org = "ddi"
bucket = "ddi"

client = InfluxDBClient(url="http://localhost:8086", token=token)
write_api = client.write_api(write_options=SYNCHRONOUS)

# RS: uZei_UmVg7IGcllVQdvKbmbCjwx5s0pe7KfTafVspsL0qGWIg6fmB34JNwWmsEGdt9aFr2Qio6ltOB9_ZrCDDw==
# SL: AW-zLqzOTpQW4sRYaKbdXpSxBLkxT8rT-RZA-IS5MYo41RZ40YoOCoNYTyu9S2La5W4KpcDzDgCfj53fk6aZuw==

In [ ]:
client.health()

### Query MongoDB, insert into InfluxDB

In [ ]:
# select box and get all measurements
col = mongo_db.list_collection_names()[0]
col

In [ ]:
id_names = mongo_db[col].find().distinct('_id')
id_names

In [ ]:
# result = mongo_db[col].find_one({},{'measurments':1}) # select first sensor with field 'measurments'
# result

In [ ]:
result = mongo_db[col].find_one({'_id':id_names[0]},{'measurments'})
result

In [ ]:
for observation in tqdm(result['measurments'].items()):
    point = Point(result['_id'].split('_')[0]) \
      .tag("sensor_name", result['_id'].split('_')[1]) \
      .field("_value", observation[1])\
      .time(datetime.strptime(observation[0], "%Y-%m-%d %H:%M:%S"), WritePrecision.S)

    write_api.write(bucket, org, point)

#### Query InfluxDB

In [ ]:
query = f'''from(bucket: "{bucket}") 
        |> range(start: -2d)'''
tables = client.query_api().query(query, org=org)